In [1]:
import json
import requests
from kafka import KafkaProducer
from datetime import datetime

# Variables de configuration
GITHUB_TOKEN = "XXXXX"  # Remplacez par votre token GitHub
KAFKA_TOPIC = "github_repos"
KAFKA_BOOTSTRAP_SERVERS = "localhost:9092"

# Initialisation du producteur Kafka
producer = KafkaProducer(
    bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# Fonction pour rechercher des dépôts GitHub créés dans un mois spécifique
def search_github_repos(month, year):
    query = f"created:{year}-{month:02d}-01..{year}-{month:02d}-30"
    url = f"https://api.github.com/search/repositories?q={query}&sort=created&order=desc&per_page=100"
    headers = {
        "Authorization": f"token {GITHUB_TOKEN}",
        "Accept": "application/vnd.github.v3+json"
    }

    repos = []
    while url:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            repos.extend(data.get("items", []))
            # Vérifiez s'il existe une page suivante
            url = response.links.get('next', {}).get('url', None)
        else:
            print(f"Erreur lors de la récupération des dépôts : {response.status_code}")
            return []

    return repos

# Fonction pour envoyer les noms des dépôts à Kafka
def send_to_kafka(repos):
    for repo in repos:
        repo_data = {
            "repo_name": repo["name"],
            "full_name": repo["full_name"],
            "html_url": repo["html_url"],
            "created_at": repo["created_at"],
            "description": repo.get("description", "Pas de description"),
            "created_at": repo["created_at"],
            "topics": repo.get("topics", [])  # Ajouter les topics ici
        }
        # Envoi des données à Kafka
        producer.send(KAFKA_TOPIC, value=repo_data)
        print(f"Dépôt envoyé à Kafka : {repo['name']}")

# Fonction principale pour récupérer et envoyer les dépôts
def collect_and_send(month, year):
    print(f"Recherche des dépôts GitHub créés en {month}/{year}...")
    repos = search_github_repos(month, year)
    if repos:
        send_to_kafka(repos)
    else:
        print(f"Aucun dépôt trouvé pour {month}/{year}.")

if __name__ == "__main__":
    try:
        # Exemple : récupérer les repos créés en novembre 2024
        collect_and_send(1, 2025)
    except KeyboardInterrupt:
        print("Arrêt du script.")
    finally:
        producer.close()

NoBrokersAvailable: NoBrokersAvailable

In [7]:
!echo $GITHUB_TOKEN


None


In [2]:
from kafka import KafkaProducer

producer = KafkaProducer(bootstrap_servers='localhost:9092')
print("KafkaProducer initialisé avec succès.")
producer.close()


KafkaProducer initialisé avec succès.
